In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from time import gmtime, strftime
from datetime import timedelta

def creatURL(idN, t):
    string = "http://ec2-54-175-179-28.compute-1.amazonaws.com/get_thinktron_data.php?device_id={}&year_month={}".format(idN,t)
    return string    

def query_data(arg1):
    r = requests.get(arg1) # URL path
    soup = BeautifulSoup(r.text,'lxml')
    a = list(soup.find_all('p'))

    # Split the list through the regular expression
    d = re.split('\s+|,|<br/>|<p>|</p>',str(a))

    # Remove the '' element from the list
    d = list(filter(lambda zz: zz != '', d)) 

    # Remove the '=' element from the list
    d = list(filter(lambda zz: zz != '=', d))

    # Remove the '[' & ']' element from the list
    try:
        d.remove(']')
        d.remove('[')
    except:
        pass
    
    return d

def calculateOFFtime(d):
    if ("No" in d) & ("results" in d):
        outputStr = "Offline over 1 month"

    else:
        # Create a dataframe from the URL by data crawling
        colName=['id', 'time', 'weather', 'air','acceleration','cleavage','incline','field1','field2','field3']
        _Num = 0
        _df  = pd.DataFrame(columns=colName)
        df   = pd.DataFrame(columns=colName)

        for ii in range(0,len(d)):    
            while colName[_Num] in d[ii]:
                _lst = d[ii + 1]
                _lst = _lst.strip(',')

                if _lst == '' or (_lst in colName):
                    _lst = None       

                _df[colName[_Num]] = [_lst] # Put the list into the dataframe
                if _Num < (len(colName)-1):
                    _Num += 1
                else:
                    df = df.append(_df, ignore_index=True)
                    _Num = 0 

        # Convert argument to a numeric type(float64 or int64)
        #numericCol = ['roll', 'pitch', 'yaw','field1','field2','field3']
        #for ii in numericCol:
        #    df[ii] = pd.to_numeric(df[ii])

        # Convert the format of date
        dates = df.time
        df.index = pd.to_datetime(dates.astype(str), format='%Y%m%d%H%M%S')
        df.index.name = 'time'
        del df['time']

        # Check dataframe format
        # df.info()

        # Query the latest time stamp
        lastestTimeStr = df.index[-1]

        # Release the memory
        del df

        # Calculate the offline time
        localTimeStamp = pd.to_datetime(strftime("%Y%m%d%H%M%S"), format="%Y%m%d%H%M%S")
        deltaT = localTimeStamp - lastestTimeStr
        alrTimeIntv = timedelta(minutes = 15)

        if deltaT > alrTimeIntv:

            deltaHr  = deltaT.seconds // 3600
            deltaMin = (deltaT.seconds % 3600) // 60
            deltaSec = deltaT.seconds % 60

            outputStr = "Offline time: {} hr, {} min, {} sec".format(deltaHr, deltaMin, deltaSec)
        else:
            outputStr = "Online"            
    return outputStr

In [24]:
idNumList  = [2015,3015,2011,1007,3014,3001,3029,3028,3022,2005,1005,3019,3021,\
              3023,3016,3025,2009,2010,1008,3017,3024,3013,3020,3018]
flag = 0
for ii in idNumList:       
    URLstr = creatURL(ii,201910) # Format in (id_Num, yyyymm)
    qD = query_data(URLstr)
    writingStr = calculateOFFtime(qD)

    now = strftime("%Y%m%d")
    queryFid = "{}_hearbeatList.txt".format(now)    
    
    if (flag == 0):
        with open(queryFid, "a") as file:
            file.write("-----Device heartbeat-----")
            file.write("\n")
            file.write("Name of project: New Taipei")
            file.write("\n")
            file.write("Query time: {}".format(strftime("%Y/%m/%d %H:%M")))
            file.write("\n")
            flag = 1   
    with open(queryFid, "a") as file:
        writing = "ID: {}    {}".format(ii,writingStr)
        file.write(writing)
        file.write("\n")
    print(str(ii) + "  Done.")

2015  Done.
3015  Done.
2011  Done.
1007  Done.
3014  Done.
3001  Done.
3029  Done.
3028  Done.
3022  Done.
2005  Done.
1005  Done.
3019  Done.
3021  Done.
3023  Done.
3016  Done.
3025  Done.
2009  Done.
2010  Done.
1008  Done.
3017  Done.
3024  Done.
3013  Done.
3020  Done.
3018  Done.


In [26]:
with open(queryFid,'r') as file:
    print(file.read())

-----Device heartbeat-----
Name of project: New Taipei
Query time: 2019/10/08 14:57
ID: 2015    Online
ID: 3015    Online
ID: 2011    Offline time: 18 hr, 12 min, 56 sec
ID: 1007    Offline time: 4 hr, 1 min, 23 sec
ID: 3014    Online
ID: 3001    Online
ID: 3029    Online
ID: 3028    Online
ID: 3022    Online
ID: 2005    Online
ID: 1005    Online
ID: 3019    Offline time: 0 hr, 15 min, 13 sec
ID: 3021    Online
ID: 3023    Online
ID: 3016    Online
ID: 3025    Online
ID: 2009    Offline time: 0 hr, 18 min, 49 sec
ID: 2010    Online
ID: 1008    Online
ID: 3017    Online
ID: 3024    Online
ID: 3013    Offline time: 18 hr, 47 min, 19 sec
ID: 3020    Online
ID: 3018    Online

